In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from tqdm import tqdm
from misc import *
import tensorflow as tf
import os 
from RNN_models import *
%load_ext autoreload
%autoreload 2

path = "/home/cooper-cooper/continous/"
itraj = 0
epochs = 20

means = np.load(path+"{}/means.npy".format(itraj), allow_pickle=True).astype(np.float32) ### this is \textbf{q}(t)
covs = np.load(path+"{}/covs.npy".format(itraj), allow_pickle=True).astype(np.float32) ## this is the \Sigma(t)
xicovs = np.load(path+"{}/xicovs.npy".format(itraj), allow_pickle=True).astype(np.float32) ## this is the \Chi(\Sigma) (evolution)
signals = np.load(path+"{}/signals.npy".format(itraj), allow_pickle=True).astype(np.float32) ##this is the dy's
A = np.load(path+"{}/A.npy".format(itraj), allow_pickle=True).astype(np.float32)
dt = np.load(path+"{}/dt.npy".format(itraj), allow_pickle=True)[0]
C = np.load(path+"{}/C.npy".format(itraj), allow_pickle=True).astype(np.float32)
D = np.load(path+"{}/D.npy".format(itraj), allow_pickle=True).astype(np.float32)

coeffs = [C, A, D , dt]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
np.linspace(10,len(signals),10).astype(np.int)

array([  10,  231,  452,  673,  894, 1115, 1336, 1557, 1778, 1999])

In [31]:

model = GaussianRecuModel(coeffs)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01))
model(sliced_dataset(signals, xicovs,1))
initial_A = model.trainable_variables


### training ####
history_A, history_loss = [], []
for time_slice in [-1]:#tqdm(np.linspace(10,len(signals),10).astype(np.int)):
    inputs = sliced_dataset(signals, xicovs,time_slice)
    histo = model.fit(x=inputs, y=inputs[1][tf.newaxis,:,:], epochs=100, verbose=1)
    for k,v in zip(histo.history["Coeffs_A"], histo.history["total_loss"]):
        history_A.append(k)
        history_loss.append(v)


#os.makedirs("/data/uab-giq/scratch/matias/quantera/trajectories/{}/".format(itraj), exist_ok=True)
np.save(path+"{}/A_history".format(itraj),np.array(history_A) )
np.save(path+"{}/loss_history".format(itraj),np.array(history_loss) )

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0181 - Coeffs_A: -0.0152
Epoch 2/100
1/1 [==============================] - 0s 815us/step - total_loss: 0.0180 - Coeffs_A: -0.0202
Epoch 3/100
1/1 [==============================] - 0s 875us/step - total_loss: 0.0180 - Coeffs_A: -0.0251
Epoch 4/100
1/1 [==============================] - 0s 819us/step - total_loss: 0.0179 - Coeffs_A: -0.0301
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0179 - Coeffs_A: -0.0350
Epoch 6/100
1/1 [==============================] - 0s 813us/step - total_loss: 0.0179 - Coeffs_A: -0.0399
Epoch 7/100
1/1 [==============================] - 0s 805us/step - total_loss: 0.0178 - Coeffs_A: -0.0447
Epoch 8/100
1/1 [==============================] - 0s 889us/step - total_loss: 0.0178 - Coeffs_A: -0.0496
Epoch 9/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0178 - Coeffs_A: -0.0543
Epoch 10/100
1/1 [==============================] - 

1/1 [==============================] - 0s 1ms/step - total_loss: 0.0167 - Coeffs_A: -0.1010
Epoch 76/100
1/1 [==============================] - 0s 2ms/step - total_loss: 0.0167 - Coeffs_A: -0.0995
Epoch 77/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0167 - Coeffs_A: -0.0979
Epoch 78/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0167 - Coeffs_A: -0.0964
Epoch 79/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0167 - Coeffs_A: -0.0949
Epoch 80/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0167 - Coeffs_A: -0.0935
Epoch 81/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0167 - Coeffs_A: -0.0921
Epoch 82/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0167 - Coeffs_A: -0.0907
Epoch 83/100
1/1 [==============================] - 0s 1ms/step - total_loss: 0.0167 - Coeffs_A: -0.0894
Epoch 84/100
1/1 [==============================] - 0s 1ms/step - to

In [30]:
history_A[-1]

array([[-0.18201381,  1.0389047 ],
       [-1.116195  , -0.02561519]], dtype=float32)